# Prepare Workspace

## Install required modules

In [1]:
#!pip install -U -q PyDrive
#!pip install geopandas

## Load required modules

In [2]:
import pandas as pd
import numpy as np

import datetime

import matplotlib.pyplot as plt
#import seaborn as sns
#from scipy import stats

#from google.colab import data_table
#data_table.enable_dataframe_formatter()
from sklearn.preprocessing import MinMaxScaler
#from sklearn.linear_model import LinearRegression
#from scipy.optimize import linprog
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error


#from pydrive.auth import GoogleAuth
#from pydrive.drive import GoogleDrive
#from google.colab import auth
#from oauth2client.client import GoogleCredentials
#from google.colab import files

/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score, cross_validate
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFECV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [4]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import StandardScaler

In [6]:
def VIF(X, threshold = 5):
  from statsmodels.stats.outliers_influence import variance_inflation_factor
  columns = X.columns.to_list()
  vif = pd.DataFrame()
  vif["VIF Factor"] = [variance_inflation_factor(X.values, i)\
                       for i in range(len(columns))]
  vif["features"] = X.columns

  ## Check for infinity VIF
  mask = (vif["VIF Factor"] == np.infty)
  if mask.any():
    to_drop=vif[mask]["features"].to_list()
    setColumns = set(columns)
    setTo_drop = set(to_drop)
    columns_new = list(setColumns.difference(setTo_drop))
    print(f"dropping due to infinite VIF:")
    print("\n".join(setTo_drop))
    return VIF(X[columns_new], threshold=threshold)

  # Drop feature with highest VIF if exceeding threshold
  if vif["VIF Factor"].max() > threshold:
    index = vif["VIF Factor"].idxmax()
    to_drop = vif.loc[index].to_dict()
    print("dropping {features} at index {index} with VIF of {VIF Factor:.2f} exceeding threshold of {threshold}"\
          .format(**to_drop, index=index, threshold=threshold))
    columns.remove(to_drop["features"])
    return VIF(X[columns], threshold)
  
  return vif

In [7]:
targets       = ['MAX_Completeness_pct', 'Weeks2MAX', 'IDX_W2M']
spatial       = ['Population Density (Per Sq. Mile)', 'Average Commute to Work (In Min)', 'AREA_SQMI', 'Density_Medical_Facilities_Population', 'Density_Medical_Facilities_Area', 'Density_Transportation_Stops_Pop', 'Density_Transportation_Stops_Area', 'Stops_per_Facility']
socioeconomic = ["% Total Population: Male","% Total Population: Female","% Total Population: Under 5 Years","% Total Population: 5 to 9 Years","% Total Population: 10 to 14 Years","% Total Population: 15 to 17 Years","% Total Population: 18 to 24 Years","% Total Population: 25 to 34 Years","% Total Population: 35 to 44 Years","% Total Population: 45 to 54 Years","% Total Population: 55 to 64 Years","% Total Population: 65 to 74 Years","% Total Population: 75 to 84 Years","% Total Population: 85 Years and Over","% Total Population: White Alone","% Total Population: Black or African American Alone","% Total Population: American Indian and Alaska Native Alone","% Total Population: Asian Alone","% Total Population: Two or More Races","% Population 25 Years and Over: Less than High School","% Population 25 Years and Over: High School Graduate or More (Includes Equivalency)","% Population 25 Years and Over: Some College or More","% Population 25 Years and Over: Bachelor's Degree or More","% Population 25 Years and Over: Master's Degree or More","% Population 25 Years and Over: Professional School Degree or More","% Population 25 Years and Over: Doctorate Degree","% Population 16 Years and Over: in Labor Force","% Population 16 Years and Over: in Labor Force: in Armed Forces","% Population 16 Years and Over: in Labor Force: Civilian","% Population 16 Years and Over: in Labor Force: Civilian: Employed","% Population 16 Years and Over: in Labor Force: Civilian: Unemployed","% Population 16 Years and Over: Not in Labor Force","% Households: Less than $25,000","% Households: $25,000 to $49,999","% Households: $50,000 to $74,999","% Households: $75,000 to $99,999","% Households: $100,000 or More","Median Household Income (In 2021 Inflation Adjusted Dollars)","Gini Index","% Workers 16 Years and Over: Car, Truck, or Van","% Workers 16 Years and Over: Drove Alone","% Workers 16 Years and Over: Carpooled","% Workers 16 Years and Over: Public Transportation (Includes Taxicab)","% Workers 16 Years and Over: Motorcycle","% Workers 16 Years and Over: Bicycle","% Workers 16 Years and Over: Walked","% Workers 16 Years and Over: Other Means","% Workers 16 Years and Over: Worked At Home","% Total: No Health Insurance Coverage","% Total: with Health Insurance Coverage","% Total: with Health Insurance Coverage: Public Health Coverage","% Total: with Health Insurance Coverage: Private Health Insurance","EP_POV150","EP_UNEMP","EP_HBURD","EP_NOHSDP","EP_UNINSUR","EP_AGE65","EP_AGE17","EP_SNGPNT","EP_LIMENG","EP_MINRTY","EP_MUNIT","EP_MOBILE","EP_CROWD","EP_NOVEH","EP_GROUPQ","F_POV150","F_UNEMP","F_HBURD","F_NOHSDP","F_UNINSUR","F_THEME1","F_AGE65","F_AGE17","F_DISABL","F_SNGPNT","F_LIMENG","F_THEME2","F_MINRTY","F_THEME3","F_MUNIT","F_MOBILE","F_CROWD","F_NOVEH","F_GROUPQ","F_THEME4","F_TOTAL",'%DAYPOP',"EP_NOINT","EP_AFAM","EP_HISP","EP_ASIAN","EP_AIAN","EP_TWOMORE","EP_OTHERRACE","%Total Child Population",'%Total University','%Total High school','%Total Less than High School','%Total Adult Population','%Total Elderly Population',"%Population with Disability","%LIMENG","%NOINT"]
others        = ["Total Population", 'Area (Land)',"Total Population:","Total Population:.1","EP_NHPI","Qualifying Name","File identification","State Postal Abbreviation","Summary Level","Geographic Component","Logical Record Number","US","Census Region","Census Division","State (Census Code)","State (FIPS Code)","County of current residence","County Subdivision (FIPS)","Place (FIPS Code)","Census Tract","Block Group","Consolidated City","American Indian Area/AlaskaNative Area/ Hawaiian Home Land(Census)","American Indian Area/AlaskaNative Area/ Hawaiian Home Land(FIPS)","American Indian Trust Land/Hawaiian Home Land Indicator","American Indian TribalSubdivision (Census)","American Indian TribalSubdivision (FIPS)","% Total Population: Some Other Race Alone","Alaska Native RegionalCorporation (FIPS)","% Total Population: Native Hawaiian and Other Pacific Islander Alone","Metropolitan and MicropolitanStatistical Area","Combined Statistical Area","Metropolitan Statistical Area-Metropolitan Division","Metropolitan Area Central City","Metropolitan/MicropolitanIndicator Flag","New England City and Town Area","New England City and TownCombined Statistical Area","New England City and Town AreaDivision","Urban Area","Urban Area Central Place","Current Congressional District ***","State Legislative District Upper","State Legislative District Lower","Voting District","ZIP Code Tabulation Area (3-digit)","5-digit ZIP Code Tabulation Area","Subminor Civil Division (FIPS)","State-School District (Elementary)","State-School District (Secondary)","State-School District (Unified)","Urban/Rural","Principal City Indicator","Traffic Analysis Zone","Urban Growth Area","Public Use Microdata Area â€“ 5%File","Public Use Microdata Area - 1% File","Geographic Identifier","Area Name","Tribal Tract","Tribal Block Group","Place (State FIPS + Place FIPS)","Total Population: Male","Total Population: Female","Total Population: Under 5 Years","Total Population: 5 to 9 Years","Total Population: 10 to 14 Years","Total Population: 15 to 17 Years","Total Population: 18 to 24 Years","Total Population: 25 to 34 Years","Total Population: 35 to 44 Years","Total Population: 45 to 54 Years","Total Population: 55 to 64 Years","Total Population: 65 to 74 Years","Total Population: 75 to 84 Years","Total Population: 85 Years and Over","Total Population:.2","Total Population: White Alone","Total Population: Black or African American Alone","Total Population: American Indian and Alaska Native Alone","Total Population: Asian Alone","EP_DISABL","Total Population: Native Hawaiian and Other Pacific Islander Alone","Total Population: Some Other Race Alone","Total Population: Two or More Races","SPL_THEME1","RPL_THEME1","E_TOTPOP","Population 25 Years and Over:","SPL_THEME2","RPL_THEME2","SPL_THEME3","RPL_THEME3","SPL_THEME4","RPL_THEME4","SPL_THEMES","E_DAYPOP","RPL_THEMES","Population 25 Years and Over: Less than High School","Population 25 Years and Over: High School Graduate or More (Includes Equivalency)","Population 25 Years and Over: Some College or More","Population 25 Years and Over: Bachelor's Degree or More","Population 25 Years and Over: Master's Degree or More","Population 25 Years and Over: Professional School Degree or More","Population 25 Years and Over: Doctorate Degree","Population 16 Years and Over:","Population 16 Years and Over: in Labor Force","Population 16 Years and Over: in Labor Force: in Armed Forces","Population 16 Years and Over: in Labor Force: Civilian","Population 16 Years and Over: in Labor Force: Civilian: Employed","Population 16 Years and Over: in Labor Force: Civilian: Unemployed","Population 16 Years and Over: Not in Labor Force","Households:","Households: Less than $25,000","Households: $25,000 to $49,999","Households: $50,000 to $74,999","Households: $75,000 to $99,999","Households: $100,000 or More","Workers 16 Years and Over:","Workers 16 Years and Over: Car, Truck, or Van","Workers 16 Years and Over: Drove Alone","Workers 16 Years and Over: Carpooled","Workers 16 Years and Over: Public Transportation (Includes Taxicab)","Workers 16 Years and Over: Motorcycle","Workers 16 Years and Over: Bicycle","Workers 16 Years and Over: Walked","Workers 16 Years and Over: Other Means","Workers 16 Years and Over: Worked At Home","Total:","Total: No Health Insurance Coverage","Total: with Health Insurance Coverage","Total: with Health Insurance Coverage: Public Health Coverage","Total: with Health Insurance Coverage: Private Health Insurance","W","ST","STATE","ST_ABBR","STCNTY","COUNTY","LOCATION","M_TOTPOP","E_HU","M_HU","E_HH","M_HH","E_POV150","M_POV150","E_UNEMP","M_UNEMP","E_HBURD","M_HBURD","E_NOHSDP","M_NOHSDP","E_UNINSUR","M_UNINSUR","E_AGE65","M_AGE65","E_AGE17","M_AGE17","E_DISABL","M_DISABL","E_SNGPNT","M_SNGPNT","E_LIMENG","M_LIMENG","E_MINRTY","M_MINRTY","E_MUNIT","M_MUNIT","E_MOBILE","M_MOBILE","E_CROWD","M_CROWD","E_NOVEH","M_NOVEH","E_GROUPQ","M_GROUPQ","MP_POV150","MP_UNEMP","MP_HBURD","MP_NOHSDP","MP_UNINSUR","MP_AGE65","MP_AGE17","MP_DISABL","MP_SNGPNT","MP_LIMENG","MP_MINRTY","MP_MUNIT","MP_MOBILE","MP_CROWD","MP_NOVEH","MP_GROUPQ","E_NOINT","M_NOINT","E_AFAM","M_AFAM","E_HISP","M_HISP","E_ASIAN","M_ASIAN","E_AIAN","M_AIAN","E_NHPI","M_NHPI","E_TWOMORE","M_TWOMORE","E_OTHERRACE","M_OTHERRACE","MP_NOINT","MP_AFAM","MP_HISP","MP_ASIAN","MP_AIAN","MP_NHPI","MP_TWOMORE","MP_OTHERRACE","Count_Medical_Facilities","Count_Transportation_Stops","EPL_POV150","EPL_UNEMP","EPL_HBURD","EPL_NOHSDP","EPL_UNINSUR","EPL_AGE65","EPL_AGE17","EPL_DISABL","EPL_SNGPNT","EPL_LIMENG","EPL_MINRTY","EPL_MUNIT","EPL_MOBILE","EPL_CROWD","EPL_NOVEH","EPL_GROUPQ"]

## Read the file

In [8]:
df_scaled=pd.read_csv('df_counties.csv')
#files.download('df_scaled.csv')

In [9]:
df_scaled

,FIPS,MAX_Completeness_pct,Weeks2MAX,IDX_W2M,Population Density (Per Sq. Mile),Area (Land),Average Commute to Work (In Min),AREA_SQMI,Density_Medical_Facilities_Population,Density_Medical_Facilities_Area,...,%Total Child Population,%Total University,%Total High school,%Total Less than High School,%Total Adult Population,%Total Elderly Population,%Population with Disability,%LIMENG,%NOINT,%DAYPOP
0,56045,-0.009041,-1.465010,-1.455969,2.873639,2398.005019,27,2398.003892,0.001152,0.003336,...,0.202003,0.546811,0.373262,0.071137,0.592512,0.205485,0.134976,0.001873,0.150533,0.730913
1,56043,-0.009041,-1.465010,-1.455969,3.469903,2238.679370,12,2238.678265,0.000882,0.003127,...,0.227600,0.681173,0.256500,0.059008,0.549691,0.222709,0.133619,0.002017,0.129333,0.918946
2,56041,-0.009041,-1.465010,-1.455969,9.854347,2081.720793,21,2081.719808,0.000834,0.008166,...,0.286292,0.565556,0.364241,0.064309,0.566394,0.147314,0.150780,0.011632,0.070089,0.982870
3,56039,-0.009041,-1.465010,-1.455969,5.834346,3996.848665,16,3996.846846,0.001113,0.006505,...,0.177966,0.770429,0.168979,0.039696,0.667267,0.154766,0.063067,0.028601,0.099032,1.277274
4,56037,-0.009041,-1.465010,-1.455969,4.072032,10426.980103,20,10426.975770,0.000461,0.001918,...,0.257778,0.592896,0.326035,0.072944,0.615276,0.126946,0.130836,0.007405,0.097043,1.040252
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3093,1009,-0.061571,-1.447915,-1.386344,91.308500,644.890660,34,644.890375,0.000554,0.049621,...,0.232372,0.480820,0.351234,0.163519,0.587885,0.179743,0.148282,0.013869,0.155606,0.730188
3094,1007,-0.061571,-1.447915,-1.386344,36.004970,622.469553,30,622.469288,0.000626,0.022491,...,0.205783,0.361961,0.439185,0.194619,0.633857,0.160361,0.167516,0.003352,0.171583,0.840216
3095,1005,-0.061571,-1.447915,-1.386344,28.540980,885.008060,26,885.007618,0.000559,0.015819,...,0.208401,0.383773,0.366769,0.243290,0.601528,0.190071,0.166187,0.017302,0.249700,1.084192
3096,1003,-0.055362,-1.593627,-1.538266,142.864400,1589.836160,26,1589.835294,0.000637,0.087430,...,0.215849,0.622558,0.273755,0.089858,0.578080,0.206071,0.140250,0.007298,0.112021,1.001457


In [10]:
df_scaled["Stops_per_Facility"].replace([-np.inf],np.nan, inplace=True)
df_scaled["Density_Medical_Facilities_Population"].replace([-np.inf],np.nan, inplace=True)
df_scaled["Density_Medical_Facilities_Area"].replace([-np.inf],np.nan, inplace=True)
df_scaled["Density_Transportation_Stops_Pop"].replace([np.inf],np.nan, inplace=True)
df_scaled["Density_Transportation_Stops_Area"].replace([np.inf],np.nan, inplace=True)

In [11]:
df_scaled[["Stops_per_Facility","Density_Medical_Facilities_Population","Density_Medical_Facilities_Area","Density_Transportation_Stops_Pop","Density_Transportation_Stops_Area"]]

,Stops_per_Facility,Density_Medical_Facilities_Population,Density_Medical_Facilities_Area,Density_Transportation_Stops_Pop,Density_Transportation_Stops_Area
0,0.0,0.001152,0.003336,0.0,0.0
1,0.0,0.000882,0.003127,0.0,0.0
2,0.0,0.000834,0.008166,0.0,0.0
3,0.0,0.001113,0.006505,0.0,0.0
4,0.0,0.000461,0.001918,0.0,0.0
...,...,...,...,...,...
3093,0.0,0.000554,0.049621,0.0,0.0
3094,0.0,0.000626,0.022491,0.0,0.0
3095,0.0,0.000559,0.015819,0.0,0.0
3096,0.0,0.000637,0.087430,0.0,0.0


In [12]:
df_scaled[np.isinf(df_scaled).any(axis=1)]

,FIPS,MAX_Completeness_pct,Weeks2MAX,IDX_W2M,Population Density (Per Sq. Mile),Area (Land),Average Commute to Work (In Min),AREA_SQMI,Density_Medical_Facilities_Population,Density_Medical_Facilities_Area,...,%Total Child Population,%Total University,%Total High school,%Total Less than High School,%Total Adult Population,%Total Elderly Population,%Population with Disability,%LIMENG,%NOINT,%DAYPOP
282,51081,-0.175545,-1.048116,-0.872572,38.83451,295.227067,23,295.22693,0.0,0.0,...,0.154993,0.358521,0.40742,0.228046,0.692979,0.152028,0.115934,0.003069,0.1675,1.027975


In [13]:
df_scaled.replace([np.inf], np.nan, inplace=True)

In [14]:
df_scaled.dropna(inplace=True)

In [15]:
df_scaled["% Total: No Health Insurance Coverage"] = df_scaled["% Total: No Health Insurance Coverage"].astype(float)/100

## Scale

In [16]:
scaler = StandardScaler()
df_scaled1=df_scaled.copy()
#scaler.fit(df_scaled[['Average Commute to Work (In Min)', "%LIMENG"]])
#scaler.fit(df_scaled[spatial+socioeconomic])
df_scaled[spatial+socioeconomic] = scaler.fit_transform(df_scaled[spatial+socioeconomic])
#df_scaled[['Average Commute to Work (In Min)', "%LIMENG"]] = scaler.transform(df_scaled[['Average Commute to Work (In Min)', "%LIMENG"]])
#df_scaled[['Average Commute to Work (In Min)', "%LIMENG"]] = scaler.transform(df_scaled[['Average Commute to Work (In Min)', "%LIMENG"]])

#df_scaled["%Total Child Population"] = df_scaled["%Total Child Population"]/100 #move later to data preparation

## Socio-economic Feature selection

In [17]:
X= df_scaled[socioeconomic+spatial]
y = df_scaled['IDX_W2M']
x= sm.add_constant(X)
slr = sm.OLS(y,X).fit()
slr_summary=slr.summary()
print(slr.summary())

                                 OLS Regression Results                                
Dep. Variable:                IDX_W2M   R-squared (uncentered):                   0.023
Model:                            OLS   Adj. R-squared (uncentered):             -0.012
Method:                 Least Squares   F-statistic:                             0.6608
Date:                Sat, 15 Apr 2023   Prob (F-statistic):                       0.997
Time:                        21:35:28   Log-Likelihood:                         -4772.2
No. Observations:                3043   AIC:                                      9750.
Df Residuals:                    2940   BIC:                                  1.037e+04
Df Model:                         103                                                  
Covariance Type:            nonrobust                                                  
                                                                                          coef    std err          t    

/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  x = pd.concat(x[::order], 1)


In [18]:
X.isna().sum().max()

0

In [19]:
np.isinf(X).sum()

% Total Population: Male                 0
% Total Population: Female               0
% Total Population: Under 5 Years        0
% Total Population: 5 to 9 Years         0
% Total Population: 10 to 14 Years       0
                                        ..
Density_Medical_Facilities_Population    0
Density_Medical_Facilities_Area          0
Density_Transportation_Stops_Pop         0
Density_Transportation_Stops_Area        0
Stops_per_Facility                       0
Length: 113, dtype: int64

## Reduce columns with VIF

In [ ]:
vif = VIF(df_scaled[socioeconomic], 5).round(2)
vif

In [ ]:
additional_features=['EP_DISABL', 
'%Population with Disability','EP_SNGPNT',
'% Population 25 Years and Over: Professional School Degree or More',
'%NOINT',
'% Workers 16 Years and Over: Carpooled']


In [ ]:
reduced_features=list(vif['features'])
reduced_features

In [ ]:
len(reduced_features)

### Feature engineering

In [ ]:
df = df_scaled[targets].copy()

In [ ]:
features=[]
for s in spatial:
  for p in (reduced_features+additional_features):
    df[s + "*" + p]= df_scaled[s].copy()*df_scaled[p].copy()
    features.append(s + "*" + p)

In [ ]:
df[reduced_features] = df_scaled[reduced_features]
df[additional_features] = df_scaled[additional_features]

In [ ]:
features += reduced_features
features += additional_features
features

In [ ]:
len(features)

In [20]:
# Define the variables
X= df_scaled[socioeconomic+spatial]
y = df_scaled['IDX_W2M']

In [21]:
# Calculate the correlation matrix
corr_matrix = X.corr().abs()

In [22]:
# Get the upper triangle of the correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

In [28]:
# Find the features with correlation greater than the threshold (0.9)
to_drop = [column for column in upper.columns if any(upper[column] > 0.6)]

In [29]:
# Drop the highly correlated features
X_reduced = X.drop(to_drop, axis=1)

In [30]:
# Get the features that were kept
kept_features = [column for column in X.columns if column not in to_drop]

In [31]:
# Print the number of features retained and dropped after removing highly correlated features
print(f"Number of features after removing highly correlated features: {X_reduced.shape[1]}")
print(f"Number of features dropped: {len(to_drop)}")

Number of features after removing highly correlated features: 39
Number of features dropped: 74


In [32]:
# Print the features that were kept
print("\nKept features:")
print(kept_features)


Kept features:
['% Total Population: Male', '% Total Population: Under 5 Years', '% Total Population: 10 to 14 Years', '% Total Population: 15 to 17 Years', '% Total Population: 18 to 24 Years', '% Total Population: 25 to 34 Years', '% Total Population: 35 to 44 Years', '% Total Population: 45 to 54 Years', '% Total Population: 85 Years and Over', '% Total Population: White Alone', '% Total Population: American Indian and Alaska Native Alone', '% Total Population: Asian Alone', '% Total Population: Two or More Races', '% Population 25 Years and Over: Less than High School', '% Population 16 Years and Over: in Labor Force: in Armed Forces', '% Population 16 Years and Over: in Labor Force: Civilian: Unemployed', '% Households: $25,000 to $49,999', '% Households: $50,000 to $74,999', '% Households: $75,000 to $99,999', 'Gini Index', '% Workers 16 Years and Over: Car, Truck, or Van', '% Workers 16 Years and Over: Carpooled', '% Workers 16 Years and Over: Motorcycle', '% Workers 16 Years a

In [33]:
# Print the features that were dropped
print("\nDropped features:")
print(to_drop)


Dropped features:
['% Total Population: Female', '% Total Population: 5 to 9 Years', '% Total Population: 55 to 64 Years', '% Total Population: 65 to 74 Years', '% Total Population: 75 to 84 Years', '% Total Population: Black or African American Alone', '% Population 25 Years and Over: High School Graduate or More (Includes Equivalency)', '% Population 25 Years and Over: Some College or More', "% Population 25 Years and Over: Bachelor's Degree or More", "% Population 25 Years and Over: Master's Degree or More", '% Population 25 Years and Over: Professional School Degree or More', '% Population 25 Years and Over: Doctorate Degree', '% Population 16 Years and Over: in Labor Force', '% Population 16 Years and Over: in Labor Force: Civilian', '% Population 16 Years and Over: in Labor Force: Civilian: Employed', '% Population 16 Years and Over: Not in Labor Force', '% Households: Less than $25,000', '% Households: $100,000 or More', 'Median Household Income (In 2021 Inflation Adjusted Doll

In [35]:
X= df_scaled[kept_features]

In [36]:
# Define the base model
base_model = RandomForestRegressor(random_state=42)

In [37]:
# Define the RFE with cross-validation
rfecv = RFECV(estimator=base_model, step=1, cv=KFold(5), scoring='r2')

In [43]:
# Fit the RFECV to the entire dataset
rfecv.fit(X, y)

RFECV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
      estimator=RandomForestRegressor(random_state=42), scoring='r2')

In [44]:
# Print the optimal number of features
print("Optimal number of features: %d" % rfecv.n_features_)

Optimal number of features: 38


In [45]:
# Get the boolean mask of retained features
retained_features_mask = rfecv.support_

In [46]:
# Get the feature names from the original dataset
feature_names = np.array(X.columns)

In [47]:
# Get the names of the retained features
retained_features = feature_names[retained_features_mask]

In [48]:
# Print the retained features
print("Retained features:")
print(retained_features)

Retained features:
['% Total Population: Male' '% Total Population: Under 5 Years'
 '% Total Population: 10 to 14 Years' '% Total Population: 15 to 17 Years'
 '% Total Population: 18 to 24 Years' '% Total Population: 25 to 34 Years'
 '% Total Population: 35 to 44 Years' '% Total Population: 45 to 54 Years'
 '% Total Population: 85 Years and Over' '% Total Population: White Alone'
 '% Total Population: American Indian and Alaska Native Alone'
 '% Total Population: Asian Alone' '% Total Population: Two or More Races'
 '% Population 25 Years and Over: Less than High School'
 '% Population 16 Years and Over: in Labor Force: in Armed Forces'
 '% Population 16 Years and Over: in Labor Force: Civilian: Unemployed'
 '% Households: $25,000 to $49,999' '% Households: $50,000 to $74,999'
 '% Households: $75,000 to $99,999' 'Gini Index'
 '% Workers 16 Years and Over: Car, Truck, or Van'
 '% Workers 16 Years and Over: Carpooled'
 '% Workers 16 Years and Over: Motorcycle'
 '% Workers 16 Years and Ov

In [49]:
# Transform the dataset to retain only the selected features
X_selected = rfecv.transform(X)

In [50]:
# Define the hyperparameter grid for the Random Forest regressor
param_grid = {
    'n_estimators': [10, 50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [51]:
# Perform a grid search with cross-validation to find the best hyperparameters
grid_search = GridSearchCV(estimator=base_model, param_grid=param_grid, cv=5, scoring='r2')
grid_search.fit(X_selected, y)

GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
             param_grid={'max_depth': [None, 10, 20, 30],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [10, 50, 100, 200]},
             scoring='r2')

In [52]:
# Train the model with the best hyperparameters on the selected features
best_model = base_model.fit(X_selected, y)

In [53]:
# Train the model with the best hyperparameters on the selected features
best_model = grid_search.best_estimator_
best_model.fit(X_selected, y)

RandomForestRegressor(max_depth=10, n_estimators=200, random_state=42)

In [54]:
# Define the evaluation metrics
scoring_metrics = ['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error']


In [55]:
# Perform cross-validation using the best model and selected features
cv_results = cross_validate(best_model, X_selected, y, cv=KFold(5), scoring=scoring_metrics)

In [56]:
# Print the cross-validation results
print("Cross-validation results:")
for metric in scoring_metrics:
    mean_score = np.mean(cv_results[f'test_{metric}'])
    if metric == 'neg_mean_squared_error':
        print(f"Mean squared error: {-mean_score:.2f}")
    elif metric == 'neg_mean_absolute_error':
        print(f"Mean absolute error: {-mean_score:.2f}")
    else:
        print(f"{metric.capitalize()}: {mean_score:.2f}")


Cross-validation results:
R2: -0.28
Mean squared error: 0.26
Mean absolute error: 0.42
